<a href="https://colab.research.google.com/github/gguex/ISH_ressources_cours_ML/blob/main/TP11_LSTM_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP 11 : LSTM et GRU

Dans ce TP, nous allons apprendre à créer et entrainer des réseaux de type LSTM et GRU pour créer des modèles génératifs.

Dans la première partie, nous allons entrainer un réseau LSTM qui permet de générer du texte "similaire" à des données d'entrainement. Nous utiliserons pour cela le premier livre des Misérables.

Dans la deuxième partie, nous allons entrainer un réseau GRU qui permet de générer des prénoms féminins ou masculins qui "sonnent juste", mais qui n'existent pas nécessairement. Nous utiliserons pour cela le jeu de données de prénoms anglophones trouvé sur http://www.cs.cmu.edu/afs/cs/project/ai-repository/ai/areas/nlp/corpora/names/ (mais vous pouvez essayer avec d'autres listes de prénoms, ces dernières sont faciles à trouver).

Les librairies nécessaires sont les suivantes :

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import spacy
import string
# Permet d'afficher le texte avec une certaine largeur
import textwrap

Enregistrons le dispositif de calcul dans une variable.

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## 1 Génération de textes avec LSTM

Dans cette partie, nous allons créer un modèle de génération de textes qui sera entrainé sur le premier livre des Misérables (cf. TP 10). Ce modèle sera constitué de plusieurs couches LSTM successives, et devra être entrainé à prédire le prochain token d'une séquence en fonction des tokens précédents.

On commence par créer une classe héritée de `torch.utils.data.Dataset` pour pouvoir générer des exemples d'entrainement.

Un entrée/sortie de ce dataset sera constitué de deux séquences, tirées de notre document, de taille `seq_len`. La séquence de sortie sera décalée d'un token sur la droite par rapport à l'entrée (voir l'exemple plus bas). Ainsi, chaque token de la séquence d'entrée sera associé avec le token suivant.

Nous allons utiliser un modèle de langage de Spacy pour tokeniser notre document.

In [3]:
class SentenceData(Dataset):
    def __init__(self, file_path, nlp, seq_len):

        # Le chemin d'accès au fichier
        self.file_path = file_path
        # La longueur des séquences d'entrée et sortie
        self.seq_len = seq_len

        # On ouvre notre fichier et on crée un object Spacy
        with open(self.file_path, "r") as f:
          text = f.read()
        doc = nlp(text)

        # On découpe notre texte en tokens, sans les espaces
        self.tokens = [token.text for token in doc if not token.is_space]

        # On enregistre le vocabulaire utilisé, et on crée des dictionnaires
        # permettant de transformer chaque token en identifiant numérique,
        # ou l'inverse.
        self.dictionary = list(set(self.tokens))
        self.id2token = {id: token for id, token in enumerate(self.dictionary)}
        self.token2id = {token: id for id, token in enumerate(self.dictionary)}

        # On transforme notre corpus en une séquence de valeurs numériques
        self.token_ids = [self.token2id[w] for w in self.tokens]

    # La taille de notre dataset est le nombre de séquences possibles
    def __len__(self):
        return len(self.token_ids) - self.seq_len

    # Un élément de notre corpus sera la séquence id et la séquence id+1.
    def __getitem__(self, id):
        return (torch.tensor(self.token_ids[id:id+self.seq_len]),
                torch.tensor(self.token_ids[id+1:id+self.seq_len+1]))

On charge le modèle de langage de Spacy.

In [4]:
# !spacy download "en_core_web_sm"
nlp = spacy.load("en_core_web_sm")

On crée une instance de notre classe `SentenceData`, en utilisant le premier livre de notre corpus et en le divisant en séquences de 10 tokens.

In [5]:
doc_path = "/content/drive/MyDrive/Colab Notebooks/ml_data/TP11/book_01.txt"
example_doc = SentenceData(doc_path, nlp, 10)

In [6]:
len(example_doc.dictionary)

4728

Notre document contient 28932 paires de séquences.

In [7]:
len(example_doc)

28932

On peut examiner un exemple en particulier, en traduisant les séquences numériques en mots.

In [8]:
sample_id = 0
input, output = example_doc[sample_id]
print(" ".join([example_doc.id2token[id.item()] for id in input]))
print(" ".join([example_doc.id2token[id.item()] for id in output]))

In 1815 , M. Charles - François - Bienvenu Myriel
1815 , M. Charles - François - Bienvenu Myriel was


On crée un objet `torch.utils.data.Dataloader` pour parcourir notre base de données, avec une taille de batch donnée.

In [9]:
batch_size = 128
my_dataloader = DataLoader(example_doc, batch_size=batch_size)

Il est temps de créer notre modèle. Ce dernier prendra notre séquence d'entrée et sera consititué de :

* Une couche d'embedding, qui transforme nos identifiants numériques en vecteurs one-hot de taille `n_vocab`, puis en vecteur de taille `embedding_dim`.
* `num_layers` couches de LSTM, avec des vecteurs d'état caché et d'état de cellule de taille `hidden_size`. On peut ajouter un dropout aux couches, afin que ces dernières n'apprennent pas le texte entièrement par coeur.
* Une couche entièrement connectée, avec `n_vocab` sorties.

La sortie de ce réseau donnera les log-odds pour les mots suivants, qui peuvent entre converties en probabilités.

Notez que la longueur de la séquence n'a pas besoin d'être précisée ici. Pytorch se chargera de prendre tous les éléments reçus, de "déplier" le réseau en fonction de leur nombre, et de calculer les sorties en transmettant les états entre les cellules.

Remarquez également que nous précisons avec `batch_first=True`, lors de la création des couches LSTM, que notre batch se situe sur la première dimension de nos tenseurs. Ainsi, Pytorch comprend que la séquence se situe sur la deuxième dimension des tenseurs (voir https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html).

In [10]:
class SeqGen(nn.Module):
    def __init__(self, n_vocab, embedding_dim, hidden_size, num_layers):
        super(SeqGen, self).__init__()

        # On sauve les paramètres dans des attributs
        self.n_vocab = n_vocab
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # La couche d'embedding
        self.embedding = nn.Embedding(
            num_embeddings=self.n_vocab,
            embedding_dim=self.embedding_dim,
        )
        # Les couches LSTM
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            dropout=0.2,
            batch_first=True
        )
        # La couche entièrement connectée
        self.lin_layer = nn.Linear(self.hidden_size, self.n_vocab)

    # La fonction foward peut prendre, en plus des entrées,
    # l'état caché et l'état de la cellule utilisé au début de la séquence.
    # Par défaut, ces états sont posés comme None.
    def forward(self, x, prev_state=None):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logodds = self.lin_layer(output)
        return logodds, state

On crée une instance de notre modèle, avec nombre de mots correspondant à notre vocabulaire, embedding de 128 et 3 couches de LSTM avec états de taille 128.

In [11]:
seq_gen = SeqGen(len(example_doc.dictionary), 128, 128, 3)

Nous allons utiliser l'entropie croisée et l'optimisateur Adam pour entrainer notre modèle.

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(seq_gen.parameters(), lr=0.001)

On entraine maintenant le modèle. Notez qu'aucun état caché est donné à notre modèle avant chaque batch, ce qui veut dire que les états initiaux seront nuls avant chaque séquence. L'entrainement est relativement simple, car la surparamétrisation n'est pas très importante pour notre modèle.

In [13]:
# Le nombre d'epochs
n_epochs = 30

# On met le modèle sur le dispositif de calcul
seq_gen.to(device)
# On le met en mode entrainement
seq_gen.train()
# On boucle sur les epochs
for epoch in range(n_epochs):

  print(f"Epoch {epoch+1}", end=": ")

  # Pour calculer la perte moyenne
  sum_loss = 0
  # On boucle sur notre dataloader
  for input, output in my_dataloader:

    # Les entrées et sorties sont mises sur le dispositif de calcul
    input = input.to(device)
    output = output.to(device)

    # On met à zéro les gradients
    optimizer.zero_grad()

    # On fait les prédictions
    pred, _ = seq_gen(input)
    # On calcule la perte, en transposant nos résultats
    loss = loss_fn(pred.transpose(1, 2), output)

    # On fait une itération de descente du gradient
    loss.backward()
    optimizer.step()

    # On ajoute la perte
    sum_loss += loss.item()

  print(f"mean loss = {sum_loss / len(my_dataloader):.4f}")

Epoch 1: mean loss = 6.7336
Epoch 2: mean loss = 6.2557
Epoch 3: mean loss = 6.1583
Epoch 4: mean loss = 6.0445
Epoch 5: mean loss = 5.8407
Epoch 6: mean loss = 5.6218
Epoch 7: mean loss = 5.4399
Epoch 8: mean loss = 5.2722
Epoch 9: mean loss = 5.1181
Epoch 10: mean loss = 4.9770
Epoch 11: mean loss = 4.8436
Epoch 12: mean loss = 4.7203
Epoch 13: mean loss = 4.5949
Epoch 14: mean loss = 4.4869
Epoch 15: mean loss = 4.3746
Epoch 16: mean loss = 4.2786
Epoch 17: mean loss = 4.1957
Epoch 18: mean loss = 4.0979
Epoch 19: mean loss = 4.0131
Epoch 20: mean loss = 3.9277
Epoch 21: mean loss = 3.8473
Epoch 22: mean loss = 3.7668
Epoch 23: mean loss = 3.6948
Epoch 24: mean loss = 3.6273
Epoch 25: mean loss = 3.5591
Epoch 26: mean loss = 3.4974
Epoch 27: mean loss = 3.4331
Epoch 28: mean loss = 3.3719
Epoch 29: mean loss = 3.3125
Epoch 30: mean loss = 3.2590


On va maintenant donner à notre modèle un début de séquence et voir comment il génère une suite. Le processus se passe en deux étapes :

* On passe une première fois la séquence initiale dans notre modèle, afin de générer le prochain token et pour récupérer l'état caché résultant.
* On va ensuite faire une boucle pour les tokens restants, en passant dans le réseau le dernier token et états obtenus à l'étape précédente.

Pour générer chaque nouveau token, on transforme les log-odds en probabilités, puis on effectue un tirage aléatoire selon ces dernières.

Le séquence finale est affichée formattée.

In [14]:
# Séquence initiale
input_sentence = "When the man"
# Nombre de tokens désiré
n_generated_tokens = 600

# --- Etape 1

# On coupe notre séquence intiale pour créer notre séquence de sortie
output_tokens = input_sentence.split()
# On met le modèle en mode évaluation
seq_gen.eval()
# Notre séquence d'entrée est transformée en identifiants numériques
input = torch.tensor([example_doc.token2id[token]
                      for token in output_tokens]).to(device)
# On passe nos entrées dans notre modèle, en ne donnant aucun état.
pred, hidden = seq_gen(input)
# On garde les logodds du dernier mot. Notez que nous utilisons
# detach().cpu() pour ne pas garder le graph de calcul et pour
# basculer le tenseur sur le cpu.
new_token_logodds = pred.detach().cpu()[-1]
# Les probabilités sont calculées avec un softmax
probs = torch.nn.functional.softmax(new_token_logodds, dim=0).numpy()
# On tire aléatoirement le token suivant, avec les probabilités précédentes
token_index = np.random.choice(len(new_token_logodds), p=probs)
# On l'ajoute à notre sortie
output_tokens.append(example_doc.id2token[token_index])

# --- Etape 2

# On boucle sur le nombre restant de tokens demandés
for i in range(n_generated_tokens - 1):
  # Notre dernier token est transformé en entrée
  input = torch.tensor([example_doc.token2id[output_tokens[-1]]]).to(device)
  # On passe notre entrée, avec les états précédants, dans notre modèle
  pred, hidden = seq_gen(input, hidden)
  # On détache, met sur le cpu et applatit la sortie
  new_token_logodds = pred.detach().cpu().flatten()
  # Les probabilités sont calculées avec un softmax
  probs = torch.nn.functional.softmax(new_token_logodds, dim=0).numpy()
  # On tire aléatoirement, avec les probabilités calculées, le token suivant
  token_index = np.random.choice(len(new_token_logodds), p=probs)
  # On l'ajoute à notre sortie
  output_tokens.append(example_doc.id2token[token_index])

# On affiche notre sortie formatée
print(textwrap.fill(" ".join(output_tokens), 79))

When the man ! It was there is a atheist for his adherence . Moreover , her
room put a politician , wears he redoubled offers little upon the remark of
instinct , with directly at the father of life , there know a heart on his
decrepit - charming ; this mystery , seemed ; as a form which is alarming , as
we have done out , opening more authorized to traits , how an excuse , rather
Cassette , although if the people to the recapitulation of purple ; this
natures francs out . While they must caused three ’s d’Assisi and perfectly ill
; with Waterloo who should been utterly wounded for him , and on charming
towards young people is a comprehend to a leisure of gold , proscribed to
stating he ! No one passed himself , frightful saying of slavery ; of other
over the house , in life , we have sometimes approaches in perfection , and
errors like him to adore children . Twenty days and reckoning ; it was very
women at her conclusion . ” The chambers steadily from the moment of touching
or God . 

---

## 2 Génération de prénoms avec GRU

Maintenant, nous allons utiliser les GRU pour créer des modèles de génération de prénoms, en utilisant des données de prénoms féminins et masculins. Le principe reste assez similaire à la partie précédente, mais nous adopterons une programmation de style plus fonctionnel.

On commence par créer une classe héritée de `torch.utils.data.Dataset` qui va donner des exemples constitués d'un nom comme entrée et du même nom décalé d'une lettre sur la droite comme sortie. Comme caractère de fin de séquence et de padding, nous allons utiliser le caractère `"\n"` déjà présent à la fin de chaque nom dans le fichier (sauf du dernier). Nous effectuons un padding sur tous les prénoms afin que tous fassent la taille du plus long prénom (pour entrainer les modèles avec des batchs).

In [15]:
class NameData(Dataset):
    def __init__(self, file_path):
        # On enregistre le chemin du fichier
        self.file_path = file_path
        # On ouvre le fichier et on lit les lignes
        with open(file_path, "r") as f:
          self.lines = f.readlines()
        # On ajoute notre caractère de padding à la dernière ligne
        self.lines[-1] += "\n"
        # La longueur maximale est la longueur de la plus grande ligne
        self.max_len = max([len(line) for line in self.lines])
        # Les lignes sont complétées avec "\n" pour avoir la même taille
        self.lines = [line + "\n"*(self.max_len - len(line))
                      for line in self.lines]
        # On met les lignes en minuscules
        self.lines_low = [line.lower() for line in self.lines]
        # On sauve le nombre de lettres différentes
        self.letters = list(set("".join(self.lines_low)))
        # On crée notre liste de nom, avec chaque lettre
        # comme élément d'une liste
        self.names = [list(line) for line in self.lines_low]

        # Ces fonctions permettent de transformer les lettres en identifiants
        # numériques, ou vice-versa.
        self.id2letter = {id: letter for id, letter in enumerate(self.letters)}
        self.letter2id = {letter: id for id, letter in enumerate(self.letters)}

        # On sauve l'identifiant du caractère de padding
        self.endl_id = self.letter2id["\n"]

        # On transforme nos listes de lettres en listes d'ids
        self.names_ids = [[self.letter2id[l] for l in name]
                          for name in self.names]

    # La longueur du dataset
    def __len__(self):
        return len(self.names_ids)

    # Un item est une entrée/sortie avec, respectivement,
    # prénom sans la dernière lettre, prénom sans la première lettre.
    def __getitem__(self, id):
        return (torch.tensor(self.names_ids[id][:-1]),
                torch.tensor(self.names_ids[id][1:]))

On crée maintenant nos instances contenant nos jeux de données.

In [16]:
female_path = "/content/drive/MyDrive/Colab Notebooks/ml_data/TP11/female.txt"
male_path = "/content/drive/MyDrive/Colab Notebooks/ml_data/TP11/male.txt"
female_data = NameData(female_path)
male_data = NameData(male_path)

Regardons un exemple.

In [17]:
input, output = female_data[0]
print([female_data.id2letter[id.item()] for id in input])
print([female_data.id2letter[id.item()] for id in output])

['a', 'b', 'a', 'g', 'a', 'e', 'l', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n']
['b', 'a', 'g', 'a', 'e', 'l', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n']


Nous créons maintenant la classe de notre modèle. Ce dernier passe  directement les vecteurs one-hot dans les couches GRU, car le nombre de caractères existants n'est pas très élevé (la dimensionnalité des vecteurs one-hot est déjà basse). Le modèle est constitué de :

* `num_layer` couches GRU avec états cachés de taille `hidden_size`.
* Une couche entièrement connectée qui envoie les états cachés finaux sur la taille du vocabulaire.

Le résultat du modèle sont les log-odds des prochains caractères.

In [18]:
class NameGen(nn.Module):
    def __init__(self, n_vocab, hidden_size, num_layers):
        super(NameGen, self).__init__()

        # On enregistre les paramètres du modèle
        self.n_vocab = n_vocab
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Les couches GRU
        self.gru = nn.GRU(
            input_size=self.n_vocab,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            batch_first=True
        )
        # La couche entièrement connectée
        self.lin_layer = nn.Linear(self.hidden_size, self.n_vocab)

    def forward(self, x, prev_state=None):
        # Cette fonction transforme les entiers en vecteurs one-hot
        one_hot = nn.functional.one_hot(x, num_classes=self.n_vocab)
        # On passe dans les couches GRU
        output, state = self.gru(one_hot.to(torch.float), prev_state)
        # Les log-odds résultantes
        logodds = self.lin_layer(output)
        return logodds, state

Nous instancions un modèle et lui passons un exemple, pour voir si tout fonctionne.

In [19]:
female_name_gen = NameGen(len(female_data.letters), 20, 2)
female_name_gen(input)[0]

tensor([[ 1.2625e-01, -1.3404e-01,  1.9273e-01,  7.4753e-02,  3.2473e-02,
         -5.3528e-02, -3.8129e-02, -2.7323e-01, -8.8352e-02, -3.2072e-02,
          8.1128e-02, -1.8301e-01, -1.1507e-01,  8.1924e-02, -1.2629e-01,
         -1.8091e-01,  2.0811e-01,  2.0678e-01,  7.6976e-02, -2.4153e-01,
          7.4301e-02, -2.5053e-01,  4.4130e-02, -2.6339e-01, -6.0192e-03,
         -2.4544e-02, -1.1928e-01,  3.2498e-02,  9.9209e-02,  3.1888e-02],
        [ 1.5580e-01, -1.3477e-01,  2.0880e-01,  7.3884e-02,  2.0569e-02,
         -8.9267e-02,  1.0815e-02, -2.9651e-01, -8.7315e-02, -9.2361e-03,
          4.0939e-02, -1.8183e-01, -1.2900e-01,  2.3720e-03, -9.1838e-02,
         -1.5401e-01,  2.3657e-01,  2.1741e-01,  7.9559e-02, -2.5496e-01,
          5.2388e-02, -2.9050e-01,  2.4339e-02, -2.9354e-01,  1.0409e-03,
         -2.2507e-02, -1.6569e-01,  4.8507e-03,  6.3490e-02,  1.3062e-02],
        [ 1.7277e-01, -1.3440e-01,  2.1694e-01,  6.5793e-02,  2.2751e-02,
         -1.1315e-01,  3.5149e-02, -

Dans l'idée de prendre une approche fonctionnelle, nous définissons ici la fonction qui permet d'entrainer un modèle.

In [20]:
def train_model(model, data, batch_size, n_epochs, device):

  # On définit la fonction de perte et l'optimisateur
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

  # On crée le dataloader
  dataloader = DataLoader(data, batch_size, shuffle=True)

  # On bascule le modèle en mode entrainement
  model.train()
  # On le met sur le dispositif de calcul
  model.to(device)
  # La boucle d'entrainement
  for epoch in range(n_epochs):

    print(f"Epoch {epoch+1}", end=": ")

    # Pour calculer la perte moyenne
    sum_loss = 0
    # La boucle sur les batchs
    for input, output in dataloader:

      # On met l'entrée et la sortie sur le dispositif de calcul
      input = input.to(device)
      output = output.to(device)

      # On met les gradients à zéro, on fait des prédictions et on
      # calcule la perte
      optimizer.zero_grad()
      pred, _ = model(input)
      loss = loss_fn(pred.transpose(1, 2), output)

      # On effectue la descente du gradient et on cumule les pertes
      loss.backward()
      optimizer.step()
      sum_loss += loss.item()

    print(f"mean Loss = {sum_loss / len(dataloader):.4f}")

On utilise maintenant la fonction créée pour entrainer un modèle avec les prénoms féminins.

In [21]:
female_name_gen = NameGen(len(female_data.letters), 30, 3)
train_model(female_name_gen, female_data, 16, 20, device)

Epoch 1: mean Loss = 1.3638
Epoch 2: mean Loss = 0.9861
Epoch 3: mean Loss = 0.9136
Epoch 4: mean Loss = 0.8787
Epoch 5: mean Loss = 0.8607
Epoch 6: mean Loss = 0.8464
Epoch 7: mean Loss = 0.8332
Epoch 8: mean Loss = 0.8210
Epoch 9: mean Loss = 0.8115
Epoch 10: mean Loss = 0.8018
Epoch 11: mean Loss = 0.7920
Epoch 12: mean Loss = 0.7805
Epoch 13: mean Loss = 0.7682
Epoch 14: mean Loss = 0.7592
Epoch 15: mean Loss = 0.7509
Epoch 16: mean Loss = 0.7446
Epoch 17: mean Loss = 0.7373
Epoch 18: mean Loss = 0.7313
Epoch 19: mean Loss = 0.7252
Epoch 20: mean Loss = 0.7194


Idem pour les prénoms masculins.

In [22]:
male_name_gen = NameGen(len(male_data.letters), 30, 3)
train_model(male_name_gen, male_data, 16, 20, device)

Epoch 1: mean Loss = 1.5468
Epoch 2: mean Loss = 1.1056
Epoch 3: mean Loss = 1.0434
Epoch 4: mean Loss = 1.0212
Epoch 5: mean Loss = 1.0033
Epoch 6: mean Loss = 0.9690
Epoch 7: mean Loss = 0.9350
Epoch 8: mean Loss = 0.9218
Epoch 9: mean Loss = 0.9139
Epoch 10: mean Loss = 0.9083
Epoch 11: mean Loss = 0.9000
Epoch 12: mean Loss = 0.8899
Epoch 13: mean Loss = 0.8797
Epoch 14: mean Loss = 0.8702
Epoch 15: mean Loss = 0.8603
Epoch 16: mean Loss = 0.8516
Epoch 17: mean Loss = 0.8439
Epoch 18: mean Loss = 0.8365
Epoch 19: mean Loss = 0.8310
Epoch 20: mean Loss = 0.8242


Créons maintenant une fonction qui permet de générer un prénom en fonction des premiers caractères. La procédure est très similaire à la génération de tokens, mais on va arrêter la génération dès que notre modèle prédit le caractère "\n", signifiant la fin de séquence.

In [23]:
def generate_name(init_letters, model, data, device):

  # --- Etape 1

  output_letters = list(init_letters)
  model.eval()
  input = torch.tensor([data.letter2id[letter]
                        for letter in output_letters]).to(device)
  pred, hidden = model(input)
  new_letter_logodds = pred.detach().cpu()[-1]
  probs = torch.nn.functional.softmax(new_letter_logodds, dim=0).numpy()
  next_letter_id = np.random.choice(len(new_letter_logodds), p=probs)
  output_letters.append(data.id2letter[next_letter_id])

  # --- Etape 2

  # On boucle tant que l'on a pas généré "\n"
  while not output_letters[-1] == "\n":
    input = torch.tensor([data.letter2id[output_letters[-1]]]).to(device)
    pred, hidden = model(input, hidden)
    new_letter_logodds = pred.detach().cpu().flatten()
    probs = torch.nn.functional.softmax(new_letter_logodds, dim=0).numpy()
    next_letter_id = np.random.choice(len(new_letter_logodds), p=probs)
    output_letters.append(data.id2letter[next_letter_id])

  return "".join(output_letters)[:-1]

Testons la fonction de génération avec un début de prénom.

In [24]:
generate_name("su", female_name_gen, female_data, device)

'sules'

Créons une liste conséquente de prénoms féminins, pour toutes les lettres de l'alphabet.

In [25]:
init_letters = string.ascii_lowercase
n_gen = 10
female_names = []
for init_letter in init_letters:
  letter_female_names = []
  for _ in range(n_gen):
    female_name = generate_name(init_letter, female_name_gen,
                                female_data, device)
    letter_female_names.append(female_name)

  female_names.extend(letter_female_names)
  print(f"{init_letter}: {', '.join(letter_female_names)}")

a: aley, alin, ally, alna, ailan, argile, aitta, aloril, ailil, aurrin
b: britte, bymcrine, blynna, bandel, bhianna, bmenni, bep, ben, byl, barlie
c: cierrie, carry, charleda, cionetto, crossia, cebline, cibarena, cabsette, cicki, chernilina
d: domyda, dara, dronna, dapty, denci, drani, damesty, danca, denia, dins
e: elvi, elly, elis, eudrandie, evenity, emarica, eis, eridis, elinesh, evee
f: feitta, ferella, flenolene, forneta, fiddi, fronny, fabhin, flemine, faresnee, forise
g: granetta, gomelah, gorciann, gortty, gaiona, goleta, gevalisa, gabry, glauriane, glivette
h: heortien, hartis, harette, harli, heytta, harlent, hilli, huda, haauna, harola
i: iie, ivhanie, id, istytha, ieandra, innica, isticfet, isbellie, inoretta, ilygmanne
j: joriegara, jiandelia, jorola, jaba, jevatey, jacque, joceana, jindala, jerviane, jistian
k: kar, kanieel, ka, kedy, karetha, kermirie, kerri, kelli, krusty, kaian
l: lufaan, loree, laume, leanjeen, lrelyh, liy, loryn, luti, lorora, lorca
m: merie, mayle

Idem pour les prénoms masculins.

In [26]:
init_letters = string.ascii_lowercase
n_gen = 10
male_names = []
for init_letter in init_letters:
  letter_male_names = []
  for _ in range(n_gen):
    male_name = generate_name(init_letter, male_name_gen,
                              male_data, device)
    letter_male_names.append(male_name)

  male_names.extend(letter_male_names)
  print(f"{init_letter}: {', '.join(letter_male_names)}")

a: axetor, adles, an, akaler, abrey, als, an, aleen, aniy, aline
b: bemdxasor, bantan, borre, barro, briilit, baunlus, barsgel, burcieluy, banje, baray
c: chysos, colley, cenmer, conry, cjedr, cawsen, cibmaugit, chackan, clallan, cont
d: digrie, danvy, duickoni, dabs, donray, dertos, dernalo, dradife, dopn, debrzet
e: engon, eckine, eudussy, eburi, erord, efbraedard, ellen, eolroth, enitollet, endarn
f: fmaelarn, fraoufderd, funne, fartin, furcy, florctard, fanie, fharride, fhillomao, fratpie
g: gimie, gareles, gerne, gannin, goscer, gestere, gaursg, gherril, gathan, gebry
h: hinfieman, harmipl, hanceo, hadmov, handanter, hlemebos, hornsem, histy, harnie, haly
i: iviel, irex, irshed, iljis, ingarnie, ilberta, ivili, ilf, ilricie, iwatort
j: jaurie, jaxel, jaatan, jembert, jeflarsond, jaston, jusothy, jomy, jark, jarion
k: kusko, kostie, kadie, kisteri, kim, koeof, kande, k, kelshethie, kace
l: largy, lunseld, ltauney, lament, lamlan, lanriko, lorny, lutf, lebbuod, letvie
m: mooune, mad

On peut maintenant regarder si notre générateur a créé des prénoms qui se trouvaient dans nos listes. Notre modèle est intéressant s'il crée quelques prénoms existants, mais pas en majorité. S'il ne crée que des prénoms existants, c'est qu'il a appris notre liste par coeur et est beaucoup trop entrainé.

In [27]:
female_found_names = [name for name in female_names
                      if name + "\n"*(female_data.max_len - len(name))
                      in female_data.lines_low]
print(textwrap.fill(", ".join(female_found_names), 79))

ally, britte, carry, dara, elly, harli, kerri, kelli, loree, reta, verla, winni


In [28]:
male_found_names = [name for name in male_names
                    if name + "\n"*(male_data.max_len - len(name))
                    in male_data.lines_low]
print(textwrap.fill(", ".join(male_found_names), 79))

kim


---